## Imports

In [1]:
import os
import pandas as pd
import json
import re

In [2]:
%run ../utils/__init__.py
%run ../utils/files.py
%run ../metrics/__init__.py

In [3]:
pd.options.display.max_columns = None

## Choose task

In [4]:
# TASK = 'seg'
# TASK = 'rg'
TASK = 'cls'

In [5]:
KEY_COLS = ['run_name', 'dataset_type']
if TASK == 'rg':
    KEY_COLS.append('free')
KEY_COLS

['run_name', 'dataset_type']

## Functions

In [6]:
TASK_FOLDER = _get_task_folder(TASK)
BASE_FOLDER = os.path.join(WORKSPACE_DIR, TASK_FOLDER)
RESULTS_FOLDER = os.path.join(BASE_FOLDER, 'results')

In [7]:
def get_suffix(filename):
    match = re.search('.*metrics-(?P<suffix>\w*)\.json', filename)
    if match is None:
        suffix = ''
    else:
        suffix = match.group('suffix')
    return suffix

In [8]:
METRIC_TYPES = [
    'chexpert',
    'grad-cam',
    'mirqi',
]

In [9]:
def load_results():
    results_by_metric_type = {}

    for run_name in os.listdir(RESULTS_FOLDER):
        if run_name == 'debug':
            continue

        folder = os.path.join(RESULTS_FOLDER, run_name)
        for filename in os.listdir(folder):
            filepath = os.path.join(folder, filename)
            if not os.path.isfile(filepath) or not filename.endswith('json'):
                continue

            metric_type = next(
                (met for met in METRIC_TYPES if met in filename),
                'base', # Default if no specific metric_type is found
            )

            with open(filepath, 'r') as f:
                results_dict = json.load(f)
   
            results_df = pd.DataFrame.from_dict(results_dict, orient='index')
            results_df.reset_index(inplace=True)
            results_df.rename(columns={'index': 'dataset_type'}, inplace=True)
            results_df['run_name'] = run_name
            if TASK == 'rg':
                results_df['free'] = get_suffix(filename)           
            
            if metric_type not in results_by_metric_type:
                results_by_metric_type[metric_type] = results_df
            else:
                prev = results_by_metric_type[metric_type]
                results_by_metric_type[metric_type] = prev.append(results_df, ignore_index=True)

    df = None
    cols_in_order = list(KEY_COLS)
    for results in results_by_metric_type.values():
        cols_in_order += [col for col in results.columns if col not in cols_in_order]
        
        if df is None:
            df = results
        else:
            df = df.merge(results, on=KEY_COLS, how='outer')
                
    return df[cols_in_order], results_by_metric_type

In [10]:
def filter_results(dataset_type=None, metrics=None,
                   metrics_contain=None, free=None,
                   contains=None, doesnt_contain=None,
                   drop=None, drop_na_rows=False, drop_key_cols=False):
    df = RESULTS_DF
    
    if dataset_type:
        if isinstance(dataset_type, str):
            df = df[df['dataset_type'] == dataset_type]
        elif isinstance(dataset_type, (list, tuple)):
            dataset_type = set(dataset_type)
            df = df[df['dataset_type'].isin(dataset_type)]
    
    if free is not None:
        free_str = 'free' if free else 'notfree'
        df = df.loc[df['free'] == free_str]
    
    if contains:
        filter_contains = lambda d, s: d.loc[d['run_name'].str.contains(s)]
        if isinstance(contains, (list, tuple)):
            for c in contains:
                df = filter_contains(df, c)
        elif isinstance(contains, str):
            df = filter_contains(df, contains)
    
    if doesnt_contain:
        filter_doesnt_contain = lambda d, s: d.loc[~d['run_name'].str.contains(s)]
        if isinstance(doesnt_contain, (list, tuple)):
            for c in doesnt_contain:
                df = filter_doesnt_contain(df, c)
        elif isinstance(doesnt_contain, str):
            df = filter_doesnt_contain(df, doesnt_contain)
    
    if drop:
        df = df.loc[~df['run_name'].str.contains(drop)]
        
    if metrics_contain:
        columns = KEY_COLS + [c for c in df.columns if metrics_contain in c]
        df = df[columns]
    elif metrics:
        columns = KEY_COLS + metrics
        df = df[columns]
    
    if drop_na_rows:
        df.dropna(axis=0, how='any', inplace=True)

    # Drop cols with all na
    df.dropna(axis=1, how='all', inplace=True)

    if drop_key_cols:
        columns = [c for c in df.columns if c == 'run_name' or c not in KEY_COLS]
        df = df[columns]
    
    return df

## Load results

In [11]:
RESULTS_DF, debug = load_results()
print(len(RESULTS_DF))
RESULTS_DF.head()

208


,run_name,dataset_type,loss,acc,hamming,bce,acc_No Finding,acc_Enlarged Cardiomediastinum,acc_Cardiomegaly,acc_Lung Lesion,acc_Lung Opacity,acc_Edema,acc_Consolidation,acc_Pneumonia,acc_Atelectasis,acc_Pneumothorax,acc_Pleural Effusion,acc_Pleural Other,acc_Fracture,acc_Support Devices,prec_No Finding,prec_Enlarged Cardiomediastinum,prec_Cardiomegaly,prec_Lung Lesion,prec_Lung Opacity,prec_Edema,prec_Consolidation,prec_Pneumonia,prec_Atelectasis,prec_Pneumothorax,prec_Pleural Effusion,prec_Pleural Other,prec_Fracture,prec_Support Devices,recall_No Finding,recall_Enlarged Cardiomediastinum,recall_Cardiomegaly,recall_Lung Lesion,recall_Lung Opacity,recall_Edema,recall_Consolidation,recall_Pneumonia,recall_Atelectasis,recall_Pneumothorax,recall_Pleural Effusion,recall_Pleural Other,recall_Fracture,recall_Support Devices,spec_No Finding,spec_Enlarged Cardiomediastinum,spec_Cardiomegaly,spec_Lung Lesion,spec_Lung Opacity,spec_Edema,spec_Consolidation,spec_Pneumonia,spec_Atelectasis,spec_Pneumothorax,spec_Pleural Effusion,spec_Pleural Other,spec_Fracture,spec_Support Devices,roc_auc_No Finding,roc_auc_Enlarged Cardiomediastinum,roc_auc_Cardiomegaly,roc_auc_Lung Lesion,roc_auc_Lung Opacity,roc_auc_Edema,roc_auc_Consolidation,roc_auc_Pneumonia,roc_auc_Atelectasis,roc_auc_Pneumothorax,roc_auc_Pleural Effusion,roc_auc_Pleural Other,roc_auc_Fracture,roc_auc_Support Devices,cm_No Finding,cm_Enlarged Cardiomediastinum,cm_Cardiomegaly,cm_Lung Lesion,cm_Lung Opacity,cm_Edema,cm_Consolidation,cm_Pneumonia,cm_Atelectasis,cm_Pneumothorax,cm_Pleural Effusion,cm_Pleural Other,cm_Fracture,cm_Support Devices,acc_Effusion,acc_Infiltration,acc_Mass,acc_Nodule,acc_Emphysema,acc_Fibrosis,acc_Pleural_Thickening,acc_Hernia,prec_Effusion,prec_Infiltration,prec_Mass,prec_Nodule,prec_Emphysema,prec_Fibrosis,prec_Pleural_Thickening,prec_Hernia,prec,recall_Effusion,recall_Infiltration,recall_Mass,recall_Nodule,recall_Emphysema,recall_Fibrosis,recall_Pleural_Thickening,recall_Hernia,recall,spec_Effusion,spec_Infiltration,spec_Mass,spec_Nodule,spec_Emphysema,spec_Fibrosis,spec_Pleural_Thickening,spec_Hernia,spec,roc_auc_Effusion,roc_auc_Infiltration,roc_auc_Mass,roc_auc_Nodule,roc_auc_Emphysema,roc_auc_Fibrosis,roc_auc_Pleural_Thickening,roc_auc_Hernia,roc_auc,pr_auc_Atelectasis,pr_auc_Cardiomegaly,pr_auc_Effusion,pr_auc_Infiltration,pr_auc_Mass,pr_auc_Nodule,pr_auc_Pneumonia,pr_auc_Pneumothorax,pr_auc_Consolidation,pr_auc_Edema,pr_auc_Emphysema,pr_auc_Fibrosis,pr_auc_Pleural_Thickening,pr_auc_Hernia,pr_auc,cm_Effusion,cm_Infiltration,cm_Mass,cm_Nodule,cm_Emphysema,cm_Fibrosis,cm_Pleural_Thickening,cm_Hernia,cm,prec_covid,prec_pneumonia,prec_normal,recall_covid,recall_pneumonia,recall_normal,spec_covid,spec_pneumonia,spec_normal,prec_COVID-19,prec_No finding,recall_COVID-19,recall_No finding,spec_COVID-19,spec_No finding,prec_Non-COVID,recall_Non-COVID,spec_Non-COVID,iou-bbox-Atelectasis,iou-bbox-Cardiomegaly,iou-bbox-Effusion,iou-bbox-Infiltration,iou-bbox-Mass,iou-bbox-Nodule,iou-bbox-Pneumonia,iou-bbox-Pneumothorax,iou-bbox-Consolidation,iou-bbox-Edema,iou-bbox-Emphysema,iou-bbox-Fibrosis,iou-bbox-Pleural_Thickening,iou-bbox-Hernia,iou-bbox,iobb-bbox-Atelectasis,iobb-bbox-Cardiomegaly,iobb-bbox-Effusion,iobb-bbox-Infiltration,iobb-bbox-Mass,iobb-bbox-Nodule,iobb-bbox-Pneumonia,iobb-bbox-Pneumothorax,iobb-bbox-Consolidation,iobb-bbox-Edema,iobb-bbox-Emphysema,iobb-bbox-Fibrosis,iobb-bbox-Pleural_Thickening,iobb-bbox-Hernia,iobb-bbox,iou-masks-Atelectasis,iou-masks-Cardiomegaly,iou-masks-Effusion,iou-masks-Infiltration,iou-masks-Mass,iou-masks-Nodule,iou-masks-Pneumonia,iou-masks-Pneumothorax,iou-masks-Consolidation,iou-masks-Edema,iou-masks-Emphysema,iou-masks-Fibrosis,iou-masks-Pleural_Thickening,iou-masks-Hernia,iou-masks,iobb-masks-Atelectasis,iobb-masks-Cardiomegaly,iobb-masks-Effusion,iobb-masks-Infiltration,iobb-masks-Mass,iobb-masks-Nodule,iobb-masks-Pneumonia,iobb-masks-Pneumothorax,iobb-masks-Consolidation,iobb-masks-Edema,iobb-masks-Emphysema,iobb-m

In [12]:
set(
    col.replace('-', '_').split('_')[0]
    for col in RESULTS_DF.columns
)

{'acc',
 'bce',
 'cm',
 'dataset',
 'hamming',
 'iobb',
 'iou',
 'loss',
 'pr',
 'prec',
 'recall',
 'roc',
 'run',
 'spec'}

## Segmentation

In [38]:
SEG_METRICS = [
    'iou', 'iou-heart', 'iou-left lung', 'iou-right lung',
    'dice', 'dice-heart', 'dice-left lung', 'dice-right lung',
]

In [24]:
replace_strs = [
    (r'^\d{4}_\d{6}_jsrt_scan_', ''),
#     ('most-similar-image', '1nn'),
#     ('_lr[\d\.]+', ''),
#     ('_size256', ''),
#     (r'_\d{4}_\d{6}_.*', ''),
#     ('dummy-', ''),
#     ('common', 'top'),
#     ('-v2', ''),
#     (r'top-(\w)\w+-(\d+)', r'top-\1-\2'),
#     ('_densenet-121', ''),
]

def rename_runs(run_name):
    s = run_name
    for target, replace_with in replace_strs:
        s = re.sub(target, replace_with, s)
    return s

In [39]:
filter_results(
    metrics=SEG_METRICS,
    dataset_type='test',
    drop='1105_180035',
).sort_values('iou')# .set_index('run_name').rename(index=rename_runs)

,run_name,dataset_type,iou,iou-heart,iou-left lung,iou-right lung,dice,dice-heart,dice-left lung,dice-right lung
5,1106_180038_jsrt_scan_lr0.0002_normS_size1024_...,test,0.859004,0.762582,0.855020,0.891177,0.920927,0.860135,0.920031,0.941517
23,1106_131033_jsrt_scan_lr0.0001_normS_size1024_wce,test,0.876946,0.768236,0.890250,0.912010,0.930650,0.861632,0.940607,0.952947
17,1106_092037_jsrt_scan_lr0.0001_normD_size1024,test,0.882099,0.772974,0.892960,0.919901,0.934353,0.867182,0.942441,0.957549
14,1106_124922_jsrt_scan_lr0.0001_normS_size1024,test,0.885864,0.773531,0.906981,0.922493,0.936624,0.867728,0.950470,0.959147
8,1106_143944_jsrt_scan_lr0.0001_normS_size1024_...,test,0.888081,0.782473,0.911865,0.918951,0.937644,0.872197,0.953031,0.957002
11,1109_122859_jsrt_scan_lr0.001_normS_size1024_w...,test,0.906357,0.804471,0.930428,0.943959,0.948732,0.888313,0.963272,0.970937
20,1106_180455_jsrt_scan_lr0.0005_normS_size1024_...,test,0.907892,0.824990,0.922734,0.935546,0.950283,0.902217,0.959132,0.966459


## Report generation

In [12]:
NLP_METRICS = ['bleu1', 'bleu2', 'bleu3', 'bleu4', 'bleu', 'rougeL', 'ciderD']
CHEXPERT_METRICS = ['acc', 'roc_auc', 'recall', 'prec', 'f1', 'roc_auc']
CHEXPERT_METRICS = [c for c in RESULTS_DF.columns
                    if any(c.startswith(ch) for ch in CHEXPERT_METRICS)]
CHEXPERT_RUNTIME_METRICS = [col for col in RESULTS_DF.columns if col.startswith('chex')]
VAR_METRICS = [c for c in RESULTS_DF.columns if 'distinct' in c]
MIRQI_METRICS = [c for c in RESULTS_DF.columns if 'MIRQI' in c]

In [29]:
ESSENTIAL_METRICS = [
    'bleu', 'ciderD', 'rougeL',
    # 'chex_f1', 'chex_acc', # 'chex_recall', 'chex_prec', # Runtime-chexpert
    # 'MIRQI-v2-f',
    'roc_auc', 'acc', 'f1', 'prec', 'recall', # Holistic-chexpert
]

In [14]:
runtime_chexpert = [c for c in RESULTS_DF.columns if c.startswith('chex_f1')]
holistic_chexpert = [c for c in RESULTS_DF.columns if c.startswith('f1')]
# metrics = runtime_chexpert + holistic_chexpert
metrics = [
    'chex_f1', 'chex_f1_No Finding',
    'f1', 'f1-No Finding'
]

In [42]:
pd.options.display.float_format = '{:.3f}'.format

In [43]:
filter_results(
    # contains='^01\d{2}_\d{6}_(h-|lstm)',
    # contains='.*dummy',
    dataset_type='test',
    free=True,
    metrics=ESSENTIAL_METRICS, # CHEXPERT_RUNTIME_METRICS,
    drop_key_cols=True,
    drop_na_rows=True,
).replace(r'^\d{4}_\d{6}_(.*)', r'\1', regex=True).sort_values('run_name')

,run_name,bleu,ciderD,rougeL,roc_auc,acc,f1,prec,recall
0,dummy-common-sentences-100_front,0.197,0.157,0.283,0.501,0.907,0.048,0.038,0.073
32,dummy-common-words-100_front,0.118,0.087,0.215,0.492,0.810,0.055,0.061,0.126
77,dummy-constant_front,0.286,0.290,0.355,0.500,0.913,0.037,0.025,0.071
8,dummy-random_front,0.196,0.142,0.275,0.498,0.887,0.063,0.058,0.071
24,h-lstm-att-v2_lr0.001_densenet-121-v2_noes_front,0.206,0.096,0.327,0.500,0.913,0.037,0.025,0.071
56,h-lstm-att-v2_lr0.001_precnn_noes_front,0.144,0.021,0.283,0.500,0.913,0.037,0.025,0.071
13,h-lstm-att-v2_lr0.001_satt_precnn_noes,0.253,0.192,0.353,0.500,0.913,0.037,0.025,0.071
61,h-lstm-v2_lr0.001_densenet-121-v2_noes_front,0.233,0.148,0.341,0.500,0.913,0.037,0.025,0.071
44,h-lstm-v2_lr0.001_precnn_noes_front,0.140,0.030,0.274,0.500,0.913,0.037,0.025,0.071
48,lstm-att-v2_lr0.001_densenet-121-v2_noes,0.187,0.109,0.305,0.500,0.915,0.037,0.025,0.071


### Compare runtime chexpert vs holistic chexpert

In [16]:
def subtract_cols(df, cols_a, cols_b, drop_na_rows=True):
    array_a = df[cols_a].to_numpy()
    array_b = df[cols_b].to_numpy()
    
    df_2 = df[KEY_COLS].copy()
    df_2 = pd.concat([df_2, pd.DataFrame(array_a - array_b, columns=cols_a)], axis=1)
    
    if drop_na_rows:
        df_2.dropna(axis=0, inplace=True, how='any')
    
    return df_2

In [27]:
metric = 'f1'

runtime_chexpert = [c for c in RESULTS_DF.columns if c.startswith(f'chex_{metric}')]
holistic_chexpert = [c for c in RESULTS_DF.columns if c.startswith(metric)]

In [28]:
df = RESULTS_DF
df = df.loc[~df['run_name'].str.contains('dummy')]
len(df)

60

In [29]:
set(df['run_name'])

{'0112_154506_lstm-v2_lr0.001_densenet-121-v2_noes',
 '0113_150104_lstm-att-v2_lr0.001_densenet-121-v2_noes',
 '0114_185612_h-lstm-v2_lr0.001_densenet-121-v2_noes_front',
 '0115_064249_h-lstm-att-v2_lr0.001_densenet-121-v2_noes_front',
 '0115_175006_h-lstm-att-v2_lr0.001_satt_densenet-121-v2_noes',
 '0118_171818_lstm-v2_lr0.001_precnn_noes_front',
 '0119_054817_lstm-att-v2_lr0.001_precnn_noes_front',
 '0119_182717_h-lstm-v2_lr0.001_precnn_noes_front',
 '0120_042616_h-lstm-att-v2_lr0.001_precnn_noes_front',
 '0120_140940_h-lstm-att-v2_lr0.001_satt_precnn_noes'}

In [30]:
df = subtract_cols(df, runtime_chexpert, holistic_chexpert)
df.head()

,run_name,dataset_type,free,chex_f1,chex_f1_No Finding,chex_f1_Enlarged Cardiomediastinum,chex_f1_Cardiomegaly,chex_f1_Lung Lesion,chex_f1_Lung Opacity,chex_f1_Edema,chex_f1_Consolidation,chex_f1_Pneumonia,chex_f1_Atelectasis,chex_f1_Pneumothorax,chex_f1_Pleural Effusion,chex_f1_Pleural Other,chex_f1_Fracture,chex_f1_Support Devices
12,0120_140940_h-lstm-att-v2_lr0.001_satt_precnn_...,train,free,0.014578,0.204091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0120_140940_h-lstm-att-v2_lr0.001_satt_precnn_...,test,free,0.011194,0.156710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0120_140940_h-lstm-att-v2_lr0.001_satt_precnn_...,val,free,0.011054,0.154756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0120_140940_h-lstm-att-v2_lr0.001_satt_precnn_...,val,notfree,0.011194,0.156710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0120_140940_h-lstm-att-v2_lr0.001_satt_precnn_...,train,notfree,0.011054,0.154756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
df.describe()

,chex_f1,chex_f1_No Finding,chex_f1_Enlarged Cardiomediastinum,chex_f1_Cardiomegaly,chex_f1_Lung Lesion,chex_f1_Lung Opacity,chex_f1_Edema,chex_f1_Consolidation,chex_f1_Pneumonia,chex_f1_Atelectasis,chex_f1_Pneumothorax,chex_f1_Pleural Effusion,chex_f1_Pleural Other,chex_f1_Fracture,chex_f1_Support Devices
count,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.0,40.0,40.0,40.0,40.000000,40.000000,40.0,40.000000,40.0
mean,0.014346,0.200229,0.001192,-0.000195,-0.000031,-0.000303,0.0,0.0,0.0,0.0,-0.000294,-0.000260,0.0,0.000505,0.0
std,0.009834,0.138644,0.005262,0.000802,0.000137,0.001017,0.0,0.0,0.0,0.0,0.001860,0.001647,0.0,0.003194,0.0
min,-0.017573,-0.246018,0.000000,-0.003636,-0.000637,-0.005913,0.0,0.0,0.0,0.0,-0.011765,-0.010417,0.0,0.000000,0.0
25%,0.011159,0.154756,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
50%,0.011846,0.157578,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
75%,0.014578,0.204091,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
max,0.044736,0.627809,0.024540,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.020202,0.0


In [32]:
df

,run_name,dataset_type,free,chex_f1,chex_f1_No Finding,chex_f1_Enlarged Cardiomediastinum,chex_f1_Cardiomegaly,chex_f1_Lung Lesion,chex_f1_Lung Opacity,chex_f1_Edema,chex_f1_Consolidation,chex_f1_Pneumonia,chex_f1_Atelectasis,chex_f1_Pneumothorax,chex_f1_Pleural Effusion,chex_f1_Pleural Other,chex_f1_Fracture,chex_f1_Support Devices
12,0120_140940_h-lstm-att-v2_lr0.001_satt_precnn_...,train,free,0.014578,0.204091,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
13,0120_140940_h-lstm-att-v2_lr0.001_satt_precnn_...,test,free,0.011194,0.156710,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
14,0120_140940_h-lstm-att-v2_lr0.001_satt_precnn_...,val,free,0.011054,0.154756,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
15,0120_140940_h-lstm-att-v2_lr0.001_satt_precnn_...,val,notfree,0.011194,0.156710,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
16,0120_140940_h-lstm-att-v2_lr0.001_satt_precnn_...,train,notfree,0.011054,0.154756,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
17,0120_140940_h-lstm-att-v2_lr0.001_satt_precnn_...,test,notfree,0.014578,0.204091,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
18,0119_054817_lstm-att-v2_lr0.001_precnn_noes_front,test,free,0.015330,0.200763,0.024540,-0.000264,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,-0.010417,0.0,0.000000,0.0
19,0119_054817_lstm-att-v2_lr0.001_precnn_noes_front,val,free,0.016680,0.237153,0.000000,-0.003636,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
20,0119_054817_lstm-att-v2_lr0.001_precnn_noes_front,train,free,0.015296,0.214141,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
21,0119_054817_lstm-att-v2_lr0.001_precnn_noes_front,test,notfree,0.010304,0.144257,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0


In [33]:
from collections import Counter

In [28]:
run_name = '0112_154506_lstm-v2_lr0.001_densenet-121-v2_noes'
debug = False
d1 = load_rg_outputs(run_name, debug=debug, free=True)
d2 = load_rg_outputs(run_name, debug=debug, free=False)
len(d1), len(d2)

(18704, 22278)

In [34]:
c1 = Counter(d1['filename'])
c2 = Counter(d2['filename'])
len(c1), len(c2)

(3826, 3826)

In [36]:
for fname in c1.keys():
    v1 = c1[fname]
    v2 = c2[fname]
    if v1 != v2:
        print('Wrong: ', fname, v1, v2)

Wrong:  2311.xml 5 6
Wrong:  2379.xml 4 6
Wrong:  2464.xml 4 6
Wrong:  2652.xml 4 6
Wrong:  2748.xml 4 6
Wrong:  28.xml 4 6
Wrong:  2843.xml 4 6
Wrong:  2871.xml 4 6
Wrong:  2951.xml 4 6
Wrong:  2952.xml 2 3
Wrong:  2955.xml 4 6
Wrong:  2981.xml 4 6
Wrong:  2986.xml 6 9
Wrong:  2999.xml 4 6
Wrong:  3010.xml 4 6
Wrong:  3040.xml 2 3
Wrong:  3057.xml 2 3
Wrong:  3064.xml 4 6
Wrong:  3170.xml 4 6
Wrong:  3256.xml 4 6
Wrong:  3388.xml 4 6
Wrong:  3398.xml 4 6
Wrong:  3409.xml 2 3
Wrong:  355.xml 4 6
Wrong:  3598.xml 4 6
Wrong:  3602.xml 4 6
Wrong:  3612.xml 4 6
Wrong:  3659.xml 4 6
Wrong:  3686.xml 4 6
Wrong:  3691.xml 4 6
Wrong:  3709.xml 6 9
Wrong:  3808.xml 4 6
Wrong:  3847.xml 4 6
Wrong:  3882.xml 4 6
Wrong:  424.xml 2 3
Wrong:  459.xml 4 6
Wrong:  498.xml 4 6
Wrong:  693.xml 4 6
Wrong:  743.xml 4 6
Wrong:  927.xml 4 6
Wrong:  944.xml 4 6
Wrong:  979.xml 4 6
Wrong:  1030.xml 4 6
Wrong:  1050.xml 4 6
Wrong:  1101.xml 6 9
Wrong:  1152.xml 4 6
Wrong:  1184.xml 4 6
Wrong:  1195.xml 4 6
Wro

Wrong:  802.xml 4 6
Wrong:  803.xml 2 3
Wrong:  1194.xml 2 3
Wrong:  1361.xml 4 6
Wrong:  1572.xml 4 6
Wrong:  2130.xml 4 6
Wrong:  2502.xml 4 6
Wrong:  2578.xml 4 6
Wrong:  2585.xml 4 6
Wrong:  3072.xml 4 6
Wrong:  3158.xml 4 6
Wrong:  525.xml 4 6
Wrong:  760.xml 4 6
Wrong:  966.xml 4 6
Wrong:  1614.xml 4 6
Wrong:  1740.xml 4 6
Wrong:  233.xml 6 9
Wrong:  2721.xml 4 6
Wrong:  3038.xml 4 6
Wrong:  3190.xml 4 6
Wrong:  702.xml 2 3
Wrong:  854.xml 4 6
Wrong:  101.xml 4 6
Wrong:  1683.xml 4 6
Wrong:  1704.xml 4 6
Wrong:  177.xml 4 6
Wrong:  189.xml 4 6
Wrong:  2092.xml 4 6
Wrong:  2961.xml 4 6
Wrong:  3188.xml 4 6
Wrong:  3435.xml 4 6
Wrong:  3490.xml 4 6
Wrong:  3926.xml 4 6
Wrong:  468.xml 4 6
Wrong:  549.xml 4 6
Wrong:  934.xml 4 6
Wrong:  2077.xml 4 6
Wrong:  2271.xml 2 3
Wrong:  2594.xml 6 9
Wrong:  2867.xml 4 6
Wrong:  1220.xml 6 9
Wrong:  1529.xml 4 6
Wrong:  2441.xml 4 6
Wrong:  3154.xml 4 6
Wrong:  3219.xml 4 6
Wrong:  3517.xml 2 3
Wrong:  3611.xml 2 3
Wrong:  3934.xml 4 6
Wrong:

In [32]:
d2.head()

,filename,epoch,dataset_type,ground_truth,generated
0,2089.xml,1,train,no active disease .,the heart size is normal .
1,2089.xml,1,train,no active disease .,the heart is normal in size
2,2435.xml,1,train,no active disease .,the heart is normal in size
3,2435.xml,1,train,no active disease .,the heart is normal in size
4,2460.xml,1,train,no active disease .,the heart is normal in size


In [31]:
set(d2['dataset_type'])

{'test', 'train', 'val'}

### Pretty-print (latex)

In [41]:
replace_strs = [
    (r'^\d{4}_\d{6}_', ''),
    ('most-similar-image', '1nn'),
    ('_lr[\d\.]+', ''),
    ('_size256', ''),
    (r'_\d{4}_\d{6}_.*', ''),
    ('dummy-', ''),
    ('common', 'top'),
    ('-v2', ''),
    (r'top-(\w)\w+-(\d+)', r'top-\1-\2'),
    ('_densenet-121', ''),
]

def rename_runs(run_name):
    s = run_name
    for target, replace_with in replace_strs:
        s = re.sub(target, replace_with, s)
    return s

In [56]:
columns = ['bleu', 'rougeL', 'ciderD'] + CHEXPERT_METRICS + MIRQI_METRICS

In [57]:
df = filter_results(dataset_type='test',
                    free=True,
                    metrics=columns,
                    contains='(?=_lstm-att-v2.*densenet|_lstm-v2.*densenet|dummy)',
                    drop='0915_173951|0915_174222|0916_104739',
                    drop_na_rows=True,
                   )

In [51]:
df

,run_name,dataset_type,free,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD,...,MIRQI-r,MIRQI-p,MIRQI-f,MIRQI-v2-r,MIRQI-v2-p,MIRQI-v2-sp,MIRQI-v2-np,MIRQI-v2-f,MIRQI-v2-attr-p,MIRQI-v2-attr-r
3,0915_174026_dummy-common-sentences-100,test,free,0.346689,0.210850,0.138440,9.576881e-02,0.197937,0.280916,0.165551,...,0.417426,0.415991,0.413998,0.310505,0.315160,0.813117,0.784744,0.311550,0.305851,0.305851
9,0915_172915_dummy-common-words-10,test,free,0.312055,0.107749,0.024166,8.550974e-07,0.110993,0.243604,0.042768,...,0.356383,0.356383,0.356383,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,0915_173524_dummy-common-sentences-10,test,free,0.275183,0.170479,0.112833,7.838136e-02,0.159219,0.294405,0.151610,...,0.457447,0.449426,0.452115,0.356383,0.356383,0.813829,0.773725,0.356383,0.356383,0.356383
21,0915_172709_dummy-constant,test,free,0.454623,0.311525,0.223393,1.653650e-01,0.288726,0.356681,0.292646,...,0.478457,0.457842,0.465502,0.356383,0.356383,0.964095,0.861018,0.356383,0.356383,0.356383
28,0918_144929_lstm-att-v2_lr0.0001_densenet-121_...,test,free,0.360850,0.226277,0.152071,1.064681e-01,0.211416,0.313536,0.186868,...,0.468323,0.467369,0.461983,0.360217,0.370789,0.896238,0.840001,0.360826,0.288435,0.284874
41,0918_125940_lstm-v2_lr0.0001_densenet-121_size256,test,free,0.379042,0.239385,0.163692,1.170487e-01,0.224792,0.337529,0.284303,...,0.448021,0.441593,0.442201,0.337433,0.341755,0.874132,0.823266,0.338564,0.321698,0.321077
58,0915_173609_dummy-random,test,free,0.362480,0.196943,0.117449,7.459662e-02,0.187867,0.263606,0.111804,...,0.431931,0.430070,0.423154,0.344548,0.343018,0.726773,0.726130,0.336981,0.175747,0.174897
107,0915_173754_dummy-common-sentences-50,test,free,0.332812,0.205903,0.137483,9.688422e-02,0.193271,0.282765,0.188344,...,0.421740,0.424866,0.420113,0.318429,0.327128,0.802019,0.777257,0.320464,0.313830,0.313830
108,0916_104837_dummy-most-similar-image_0717_1202...,test,free,0.382749,0.220256,0.142339,9.958416e-02,0.211232,0.288225,0.230329,...,0.435919,0.444452,0.433369,0.347119,0.355474,0.754784,0.753767,0.345257,0.186351,0.185880
115,0915_173307_dummy-common-words-100,test,free,0.348717,0.089660,0.016369,6.481232e-07,0.113687,0.206325,0.071575,...,0.224643,0.209504,0.200727,0.198094,0.160554,0.207445,0.328014,0.168722,0.059774,0.059563


In [59]:
shorten_cols = lambda s: s.replace('MIRQI-v2', 'v2')

In [61]:
print(df.set_index('run_name').rename(
    index=rename_runs,
    columns=shorten_cols,
).sort_index().to_latex(
    columns=[shorten_cols(c) for c in columns],
    float_format='%.3f',
    column_format='l' + 'c' * len(columns),
))

\begin{tabular}{lccccccccccccccc}
\toprule
{} &  bleu &  rougeL &  ciderD &   acc &  roc\_auc &  MIRQI-r &  MIRQI-p &  MIRQI-f &  v2-r &  v2-p &  v2-sp &  v2-np &  v2-f &  v2-attr-p &  v2-attr-r \\
run\_name  &       &         &         &       &          &          &          &          &       &       &        &        &       &            &            \\
\midrule
1nn       & 0.211 &   0.288 &   0.230 & 0.903 &    0.518 &    0.436 &    0.444 &    0.433 & 0.347 & 0.355 &  0.755 &  0.754 & 0.345 &      0.186 &      0.186 \\
constant  & 0.289 &   0.357 &   0.293 & 0.915 &    0.500 &    0.478 &    0.458 &    0.466 & 0.356 & 0.356 &  0.964 &  0.861 & 0.356 &      0.356 &      0.356 \\
lstm      & 0.225 &   0.338 &   0.284 & 0.912 &    0.505 &    0.448 &    0.442 &    0.442 & 0.337 & 0.342 &  0.874 &  0.823 & 0.339 &      0.322 &      0.321 \\
lstm-att  & 0.211 &   0.314 &   0.187 & 0.918 &    0.508 &    0.468 &    0.467 &    0.462 & 0.360 & 0.371 &  0.896 &  0.840 & 0.361 &      0.288 &  

## Classification

In [14]:
# contains = 'covid-x'
# contains = 'cxr14'
# contains = 'e0'
# contains = '0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid'
# contains = '0717_101812_covid-x_densenet-121_lr1e-06_os-max2_aug-covid'
run_name = '0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid' # WINNER

contains = '0717_101812_covid-x_densenet-121_lr1e-06_os-max2_aug-covid'
contains = 'covid-uc'

In [15]:
ESSENTIAL_METRICS = [
    'acc', 'roc_auc', 'hamming', # 'prec', 'recall',
]

In [27]:
metrics = [
    'acc', 'roc_auc', 'prec', 'recall', 'roc_auc_Cardiomegaly', 'roc_auc_Pneumonia',
    'recall_Cardiomegaly', 'recall_Pneumonia',
    'iobb-masks', 'iobb-masks-Cardiomegaly', 'iobb-masks-Pneumonia',
]

In [14]:
pd.options.display.float_format = '{:.3f}'.format

In [37]:
def simplify_names(s):
    model_name = re.sub(r'^\d{4}_\d{6}_cxr14_(.*)_lr.*', r'\1', s)
    if 'hint' in s:
        return f'{model_name}_hint'
    return model_name

In [39]:
d = filter_results(
    contains=r'^(01|12)\w+_cxr14_',
    doesnt_contain=[
        r'_Card',
        r'_Pneu',
        '1027_144914', # really bad results
    ],
    dataset_type='test-bbox', # -bbox
    # metrics_contain='iou',
    metrics=metrics, # ESSENTIAL_METRICS,
    drop_key_cols=True,
    # drop_na_rows=True,
).sort_values('run_name')
d['run_name'] = d['run_name'].apply(simplify_names)
d

/home/pdpino/medical-ai/pyenv3.6/lib/python3.6/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,run_name,iobb-masks,iobb-masks-Cardiomegaly,iobb-masks-Pneumonia
196,mobilenet-v2_hint,0.458,0.521,0.461
198,densenet-121-v2,0.173,0.257,0.204
201,resnet-50-v2,0.172,0.360,0.126
193,mobilenet-v2,0.189,0.258,0.185


## Report-generation: results at different report lengths

In [32]:
vals_words = [20, 25, 27, 33, 44, None]
vals_sents = [3, 4, 5, 6, None]

In [40]:
max_words = vals_words[0]
suffix = f'max-words-{max_words}' if max_words else ''
all_results = load_results(suffix)
results_df_test = create_results_df(all_results, 'test')
results_df_test

,loss,word_acc,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD
0717_041434_lstm_lr0.0001_densenet-121,5.579509,0.149129,0.478934,0.325275,0.256634,0.218300,0.319786,0.395486,1.354237
0716_211601_lstm-att_lr0.0001_densenet-121,7.382415,0.068293,0.442359,0.282490,0.199632,0.150046,0.268632,0.358223,0.775244
0717_015057_h-lstm_lr0.0001_densenet-121,4.548621,0.162518,0.328184,0.195510,0.126410,0.082401,0.183126,0.334782,0.256378
0716_234501_h-lstm-att_lr0.0001_densenet-121,5.157984,0.132504,0.298223,0.179060,0.115951,0.075574,0.167202,0.315891,0.101786
